In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import L2

In [ ]:
splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/winvoker/turkish-sentiment-analysis-dataset/" + splits["train"])
df = df.drop(["dataset"] , axis=1)
df_Positive = df[df['label']=='Positive']
df_Negative = df[df['label']== 'Negative']
df_Notr = df[df['label']=='Notr']
df_Notr_az = df_Notr.sample(50000)
df_Positive_az = df_Positive.sample(50000)
df_Negative_az = df_Negative.sample(50000)
df = pd.concat([df_Positive_az, df_Notr_az, df_Negative_az])

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

texts = df['text'].tolist()
sentiments = df['label'].tolist()


tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100 
X = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=max_length)

sentiment_map = {'Positive': 0, 'Notr': 1, 'Negative': 2}
y = [sentiment_map[sentiment] for sentiment in sentiments]
y = to_categorical(y)

X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

with tf.device('/GPU:0'): 
    inputs = Input(shape=(max_length,))
    x = Embedding(vocab_size, 128)(inputs)
    x = LSTM(64, return_sequences=True)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(3, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

with tf.device('/GPU:0'): 
    history = model.fit(X, y, epochs=3, batch_size=8, validation_split=0.2, verbose=1)

Num GPUs Available:  1
GPU memory growth enabled
Epoch 1/3
15000/15000 [==============================] - 243s 16ms/step - loss: 0.2309 - accuracy: 0.9135 - val_loss: 0.4171 - val_accuracy: 0.8232
Epoch 2/3
15000/15000 [==============================] - 213s 14ms/step - loss: 0.0992 - accuracy: 0.9655 - val_loss: 0.5504 - val_accuracy: 0.7954
Epoch 3/3
15000/15000 [==============================] - 223s 15ms/step - loss: 0.0448 - accuracy: 0.9861 - val_loss: 0.5367 - val_accuracy: 0.8122


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

texts = df['text'].tolist()
sentiments = df['label'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
X = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=max_length)

sentiment_map = {'Positive': 0, 'Notr': 1, 'Negative': 2}
y = [sentiment_map[sentiment] for sentiment in sentiments]
y = to_categorical(y)

X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

with tf.device('/GPU:0'): 
    inputs = Input(shape=(max_length,))
    x = Embedding(vocab_size, 128)(inputs)
    x = LSTM(64, return_sequences=True)(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation='relu', kernel_regularizer=L2(0.01))(x)
    x = Dropout(0.6)(x)
    outputs = Dense(3, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)


initial_learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

with tf.device('/GPU:0'):
    history = model.fit(
        X, y,
        epochs=10,
        batch_size=8,
        validation_split=0.2,
        verbose=1,
        callbacks=[lr_scheduler, early_stopping]
    )

Num GPUs Available:  1
GPU memory growth enabled
Epoch 1/3
15000/15000 [==============================] - 252s 17ms/step - loss: 0.2864 - accuracy: 0.9056 - val_loss: 0.5891 - val_accuracy: 0.7764 - lr: 0.0010
Epoch 2/3
15000/15000 [==============================] - 214s 14ms/step - loss: 0.1352 - accuracy: 0.9610 - val_loss: 0.5101 - val_accuracy: 0.8236 - lr: 0.0010
Epoch 3/3
15000/15000 [==============================] - 214s 14ms/step - loss: 0.0807 - accuracy: 0.9804 - val_loss: 0.7766 - val_accuracy: 0.7981 - lr: 0.0010


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

texts = df['text'].tolist()
sentiments = df['label'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1

max_length = 100 
X = pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=max_length)

sentiment_map = {'Positive': 0, 'Notr': 1, 'Negative': 2}
y = [sentiment_map[sentiment] for sentiment in sentiments]
y = to_categorical(y)

X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

with tf.device('/GPU:0'):
    inputs = Input(shape=(max_length,))
    x = Embedding(vocab_size, 128)(inputs)
    x = GlobalAveragePooling1D()(x)
    x = Dense(32, activation='relu', kernel_regularizer=L2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(3, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

initial_learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

with tf.device('/GPU:0'):
    history = model.fit(
        X, y,
        epochs=10,
        batch_size=8,
        validation_split=0.2,
        verbose=1,
        callbacks=[lr_scheduler, early_stopping]
    )

Num GPUs Available:  1
GPU memory growth enabled
Epoch 1/10
15000/15000 [==============================] - 185s 12ms/step - loss: 0.4688 - accuracy: 0.8419 - val_loss: 0.9225 - val_accuracy: 0.6320 - lr: 0.0010
Epoch 2/10
15000/15000 [==============================] - 156s 10ms/step - loss: 0.2743 - accuracy: 0.9163 - val_loss: 0.5710 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 3/10
15000/15000 [==============================] - 156s 10ms/step - loss: 0.2240 - accuracy: 0.9334 - val_loss: 0.7536 - val_accuracy: 0.7485 - lr: 0.0010
Epoch 4/10
15000/15000 [==============================] - 160s 11ms/step - loss: 0.1984 - accuracy: 0.9438 - val_loss: 0.4088 - val_accuracy: 0.8800 - lr: 0.0010
Epoch 5/10
15000/15000 [==============================] - 160s 11ms/step - loss: 0.1844 - accuracy: 0.9483 - val_loss: 0.5803 - val_accuracy: 0.8332 - lr: 0.0010
Epoch 6/10
15000/15000 [==============================] - ETA: 0s - loss: 0.1680 - accuracy: 0.9529
Epoch 6: ReduceLROnPlateau reducing learn

In [ ]:
text = "Turkcell normal"
predict_sentiment(text)

1/1 [==============================] - 0s 37ms/step


'nötr'

In [ ]:
model.save('sentimentmodel')

In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)